In [ ]:
from ipynb.fs.defs.ntt_invntt import barrett_reduce
def poly_add(a:list, b:list) -> list:
    assert(len(a)==len(b))
    r = []
    for i in range(len(a)):
        # print(a[i] + b[i], end=', ')
        r.append(int(a[i] + b[i]))
    return r

def poly_reduce(a:list) -> list:
    r = [0]*len(a)
    for i in range(len(a)):
        r[i] = barrett_reduce(a[i])
    return r

def check(a,b):
    print("[+] checking...")
    assert(len(a)==len(b))
    for i in range(len(a)):
        if(a[i]!=b[i]):
            print(i,"a: ",a[i], "b: ",b[i])

In [1]:
from ipynb.fs.defs.ntt_invntt import fqmul
def basemul(AA:list, b:list, zeta:int) -> list:
    r = [0,0]
    r[0]  = fqmul(AA[1], b[1])
    r[0]  = fqmul(r[0], zeta)
    r[0] += fqmul(AA[0], b[0])
    r[1]  = fqmul(AA[0], b[1])
    r[1] += fqmul(AA[1], b[0])
    return r

a = [123,321]
b = [456,654]
zeta = 1000
basemul(a,b,zeta)

[646, -1193]

In [2]:
from ipynb.fs.defs.ntt_invntt import ntt, barrett_reduce
# 
def poly_ntt(AA:list) -> list:
    b = ntt(AA)
    for i in range(len(b)):
        b[i] = barrett_reduce(b[i])
    return b

In [ ]:
from ipynb.fs.defs.cbd2 import cbd2_hash
from ipynb.fs.defs.test import convert_mem

# HASH -> CBD -> NTT -> POLYVEC RAM A

m = '02030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f2021'
c1 = cbd2_hash(m,0)
c2 = cbd2_hash(m,1)
c3 = cbd2_hash(m,2)

n1 = poly_ntt(c1)
n2 = poly_ntt(c2)
n3 = poly_ntt(c3)

t1 = convert_mem("../sim_output/tb_kyber_pke_enc-polyvec-RAM_A-k_0.txt")
t2 = convert_mem("../sim_output/tb_kyber_pke_enc-polyvec-RAM_A-k_1.txt")
t3 = convert_mem("../sim_output/tb_kyber_pke_enc-polyvec-RAM_A-k_2.txt")

# print(t1)
# print(n1)

check(t1,n1)
check(t2,n2)
check(t3,n3)

print("[+] done")

[+] checking...
[+] checking...
[+] checking...
[+] done


In [4]:
from ipynb.fs.defs.cbd2 import cbd2_hash
from ipynb.fs.defs.ntt_invntt import invntt

from zetas import zetas

def poly_basemul_montgomery(a:list, b:list) -> list:
    r = []
    # print(a[0:2])
    for i in range(int(256/4)):
        ii = i*4
        r1 = basemul(a[ii  :ii+2], b[ii  :ii+2], zetas[64+i]     )
        r2 = basemul(a[ii+2:ii+4], b[ii+2:ii+4], zetas[64+i] * -1)
        for t in r1:
            r.append(t)
        for t in r2:
            r.append(t)
    return r

# function here
def polyvec_basemul_acc_mont(b1:list,b2:list,b3:list,m:str,s1:int,s2:int,s3:int)-> list:
    n1 = poly_ntt(cbd2_hash(m,s1))
    n2 = poly_ntt(cbd2_hash(m,s2))
    n3 = poly_ntt(cbd2_hash(m,s3))
    # print(n1)
    # print(n2)
    # print(n3)
    r1 = poly_basemul_montgomery(n1,b1)
    r2 = poly_basemul_montgomery(n2,b2)
    r3 = poly_basemul_montgomery(n3,b3)
    r = []
    for i in range(len(r1)):
        temp = r1[i] + r2[i] + r3[i]
        temp = barrett_reduce(temp)
        r.append(temp)
    return r

In [5]:
from ipynb.fs.defs.test import convert_mem

m = '02030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f2021'
b1 = []
b2 = []
b3 = []
for i in range(256):
    b1.append(i)
for i in range(256,256*2):
    b2.append(i)
for i in range(256*2,256*3):
    b3.append(i)

output1 = convert_mem("../sim_output/tb_kyber_pke_enc-invntt-ram1.txt")
r = polyvec_basemul_acc_mont(b1,b2,b3,m,0,1,2)
# check(output1, r)
# print(output1)
# for i in range(len(r)):
#     print(r[i], end=', ')
#     if(i%8==7):
#         print()
output2 = convert_mem("../sim_output/tb_kyber_pke_enc-invntt-ram2.txt") 
# output3 is C implementation calculation
output3 = [
569, -900, -513, 460, 1421, 334, -1619, 1379, 
-618, 247, 1088, 1440, 900, -898, -616, 450, 
-533, 846, 856, 323, -1667, 1107, -1242, -349, 
1075, 1122, -1427, 1327, 1420, 1119, -670, -373, 
-6, 40, -667, -34, 1355, -962, -47, -1613, 
-495, 1616, 892, 771, -930, -575, -133, 350, 
100, 46, -1519, -822, 3, 79, 668, -266, 
-1242, -1540, -195, 320, 1571, -255, -106, 1431, 
1596, -1031, 412, -1540, -456, -289, -1239, -452, 
-491, -1052, -1028, 1442, 221, 71, -557, 1225, 
-249, 520, 1642, 374, 102, 1246, -1575, -415, 
1106, -911, 396, -1535, 326, -233, 1363, -597, 
-14, 1464, -766, -857, -1543, 1428, 1291, 1359,
-1602, 907, -923, 407, 105, -981, 696, -192, 
-577, -355, 479, 277, 1636, -1297, 1181, 539,
-1425, -25, -1617, -1216, -1553, 497, 1239, -519,
-725, -235, -878, -355, 255, 1219, 634, -775,
288, -814, -465, 1159, 208, 771, -379, 789, 
122, 1355, 225, -1236, -1599, 396, -637, -1456,
928, 701, -212, 1151, -159, -709, -1675, 733,
-1640, 855, 1227, -440, -906, 673, -1100, 731, 
877, -1455, 245, 1026, -611, 217, 702, 156,
1238, -1550, -1543, -1073, 338, 193, -1443, -1118,
-990, 55, -1029, -128, -1114, -1165, -139, 930, 
1110, -455, -1539, 1677, 839, 331, -1336, -502,
-1236, -1488, -605, 1216, 1047, 1390, -278, -119,
-468, -660, -994, 903, 737, 715, 1151, 1348,
-75, 1259, -1220, 1097, -733, 680, -664, -330, 
-861, 1653, 538, -1059, 860, -28, 1024, 164,
-303, 118, -663, -1061, -334, -956, 1506, -1282,
438, -900, 1201, -572, 1213, -1247, -869, -285,
1489, -1395, 672, 989, 1109, 374, 336, -648
]
rr = invntt(r)
print(rr)
check(output3,rr) # this does not give discrepancy, meaning my python code is correct
check(output2,rr) # this gives discrepancy, meaning the invntt verilog module is slightly different
# it is however an congruence difference, the rest of the operations are addition, so for now it's fine
# I guess if modular arithmetic is well-defined, it's fine?

[569, -900, -513, 460, 1421, 334, -1619, 1379, -618, 247, 1088, 1440, 900, -898, -616, 450, -533, 846, 856, 323, -1667, 1107, -1242, -349, 1075, 1122, -1427, 1327, 1420, 1119, -670, -373, -6, 40, -667, -34, 1355, -962, -47, -1613, -495, 1616, 892, 771, -930, -575, -133, 350, 100, 46, -1519, -822, 3, 79, 668, -266, -1242, -1540, -195, 320, 1571, -255, -106, 1431, 1596, -1031, 412, -1540, -456, -289, -1239, -452, -491, -1052, -1028, 1442, 221, 71, -557, 1225, -249, 520, 1642, 374, 102, 1246, -1575, -415, 1106, -911, 396, -1535, 326, -233, 1363, -597, -14, 1464, -766, -857, -1543, 1428, 1291, 1359, -1602, 907, -923, 407, 105, -981, 696, -192, -577, -355, 479, 277, 1636, -1297, 1181, 539, -1425, -25, -1617, -1216, -1553, 497, 1239, -519, -725, -235, -878, -355, 255, 1219, 634, -775, 288, -814, -465, 1159, 208, 771, -379, 789, 122, 1355, 225, -1236, -1599, 396, -637, -1456, 928, 701, -212, 1151, -159, -709, -1675, 733, -1640, 855, 1227, -440, -906, 673, -1100, 731, 877, -1455, 245, 1026, -6

A*y:  0 [879, 1623, 1340, 866, -222, -459, 1350, 1394, -796, -134, -728, -696, -1174, 401, -1367, -1104, -1484, -1168, 839, 1091, 727, 328, -459, -354, -1543, 1047, -83, -966, 458, 951, -358, 210, -1377, -437, -620, -1267, 273, 975, -318, 984, 469, 1539, 1025, -1184, 1279, -983, 1014, 928, -5, -1108, 1184, 1313, -1647, 1512, 916, 976, 933, -484, -810, -79, -1637, 180, -212, -923, -863, 1104, -1378, -410, -697, 534, 886, 1499, -604, 920, 808, 520, -1663, -1569, 857, 1575, -639, 1506, 1389, 542, 471, -611, 448, 265, -1248, -541, 917, 1609, 282, 521, -766, -393, 283, -624, -195, 984, -561, -209, -102, -761, 963, -943, 1134, -1518, -1572, -1527, 128, 734, -594, 1224, -574, -24, -320, 262, 739, -300, 1073, 63, 56, 961, 1482, 474, 568, -533, 548, -781, 398, -1223, -1531, 46, -792, 132, 1430, -1374, 959, -224, -1416, 1028, -1117, 1134, 380, 1394, 894, 399, -1385, -1352, -231, -355, -1573, -45, 257, -1417, 1075, 145, -345, 984, 514, 1187, 301, 253, -1317, -1133, -1108, -1567, 1307, 325, -1049,

In [7]:
from ipynb.fs.defs.test import convert_mem
from ipynb.fs.defs.checking import check_dual

output1 = convert_mem("../sim_output/tb_kyber_pke_enc-polyvec-RAM_C-1.txt")
output2 = convert_mem("../sim_output/tb_kyber_pke_enc-polyvec-RAM_C-2.txt")

output = convert_mem("../sim_output/tb_kyber_pke_enc-invntt-ram2.txt")

output3 = convert_mem("../sim_output/tb_kyber_pke_enc-accu1-ram_a-k_0.txt")
output4 = convert_mem("../sim_output/tb_kyber_pke_enc-accu1-ram_b-k_0.txt")

# print(Intt_Ay[0])
# check(Intt_Ay[0], output)
# check_dual(accu1[0],output3,output4)